In [182]:
import sys 
import os 
#sys.path.append('../../')
sys.path.append('../')

import numpy as np
from numpy import concatenate
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.cluster import KMeans
import warnings
import os
warnings.filterwarnings("ignore")

from utils import utilidades as myutils
import math

In [183]:
def clean_group_data(dataset, parameter, range_upper = 1,range_lower = 1):
    
    q1, q3 = np.percentile(dataset[parameter],[25,75])
    iqr = q3 - q1
    lower_bound = q3 - range_upper*(1.5 * iqr)  
    upper_bound = q3 + range_upper*(1.5 * iqr)  

    # Deleting lower bound and upper bound from the dataset LinkTT2
    dataset = dataset.loc[(dataset[parameter] >= lower_bound) & 
                                              (dataset[parameter] <= upper_bound)]
    #dataset.interpolate()
        
    return dataset

def resultado(df,tecnica,parameter,amostras,metrica,lag):
    
    df.loc[len(df)]= [tecnica,parameter,amostras,round(metrica,2),lag]
    
    
    return df

#Prepara o conjunto de dados em X e y, considerando a janela de visualização (lags).
#cy = coluna que será predita
def prepara_dados(dados,lags,cy):
    X = []
    y = []
    for i in range(lags, len(dados)):
        X.append(dados[i-lags:i,:])
        y.append(dados[i, cy])
       
    return np.array(X), np.array(y)

def pegar_dados_coluna_predita_train_test(trainingd,percent,index_coluna):
    data = trainingd.iloc[:,:].values
    train = trainingd.iloc[0:int(len(data)*percent),:].values
    train_previsao = trainingd.iloc[0:int(len(data)*percent),index_coluna:index_coluna+1].values
    test = trainingd.iloc[len(train):,index_coluna:index_coluna+1].values 
    '''
    print('Nº observações:', len(data))
    print('treino:',len(train))
    print('teste:',len(test))
    
    '''
    
    return train,train_previsao, test

#Normalização dos dados: Normaliza os dados dentro um intervalo (0 a 1).
def normalizacao(train,test):
    sc = MinMaxScaler()
    testd = test
    train = sc.fit_transform(train)
    test = sc.fit_transform(test)    
    return train,test,testd

def tranformacao_log(valores_parametro):
   
    valores_log = []
   
    for i in range(len(valores_parametro)):
        #print(np.log(np.array(valores_parametro[i]) + 0.000000001))
        valores_log.append(np.log(np.array(valores_parametro[i]) + 0.000000001))
       
    return valores_log
    

In [184]:
parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    DFmerged = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  
    
    print(parameter)
     
    DFmerged = clean_group_data(DFmerged, 'valor')
    
    DFmerged['data_coleta'] = pd.to_datetime(DFmerged['data_coleta'])
    DFmerged.index = DFmerged['data_coleta']    
    DFmerged = DFmerged.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged = DFmerged.loc['1979-01-31':]
    
    DFmerged = DFmerged.interpolate(method = 'pad')
    
    DFmerged[parameter] = tranformacao_log(DFmerged[parameter])
        
    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
    #seleciona os dados
    
    train,train_previsao, test = pegar_dados_coluna_predita_train_test(DFmerged,0.70,0)
    y_test = test
        
    for lags in range(1,11):
        
        #normalização dos dados
        scaler = MinMaxScaler()
        train,test,testd = normalizacao(train,test)
                
        #Prepara os dados de treinamento
        train_X,train_y = prepara_dados(train, lags,0)  
        train_X_lstm = train_X
        train_y_lstm = train_y
                
        #Ajusta a dimensão de train_X 
        nsamples, nx, ny = train_X.shape
        train_X = train_X.reshape((nsamples,nx*ny))
    
        #Prepara os dados de teste
        entradas = DFmerged[len(DFmerged) - len(test) - lags:].values
        entradas = scaler.fit_transform(entradas)  
    
        test_X = []
        for i in range(lags, lags+len(test)):        
            test_X.append(entradas[i-lags:i, 0:1])
        test_X = np.array(test_X)
        
        test_X_lstm = test_X
                
        #Ajusta a dimensão de test_X e train_X
        nsamples, nx, ny = test_X.shape
        test_X = test_X.reshape((nsamples,nx*ny))
                        
        '''
        dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
        dataset.columns = ['X','y']
        dataset.dropna(inplace=True)
        '''
                        
        #Regressão Linear
        model = LinearRegression(normalize=False)

        model.fit(train_X,train_y)
        
        #Dados de teste
        previsoes = model.predict(test_X).reshape(-1, 1) 
        previsoes = scaler.inverse_transform(previsoes)
        
        #print('y_test:',len(y_test))
        #print('previsoes:',len(previsoes))
               
        df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_test[:len(previsoes)], previsoes),lags)
        
        
        #Random Forest
        model = RandomForestRegressor()

        model.fit(train_X,train_y)
        
        #Dados de teste
        previsoes = model.predict(test_X).reshape(-1, 1)
        previsoes = scaler.inverse_transform(previsoes)

        df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_test[:len(previsoes)], previsoes),lags)
        
        #MLP
        model = Sequential()
        model.add(Dense(units = 10, activation = 'relu', input_dim = train_X.shape[1]))
        model.add(Dense(units = 21, activation = 'relu'))
        model.add(Dense(units = 1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X,train_y, validation_data = (train_X,train_y),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(test_X)
            previsoes = scaler.inverse_transform(previsoes)

            media_previsoes.append(myutils.mean_absolute_percentage_error(y_test[:len(previsoes)], previsoes))  

        df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged),np.mean(media_previsoes),lags)

        
        #LSTM

        model = Sequential()
        model.add(LSTM(units = 10, input_shape = (train_X_lstm.shape[1], 1)))
        model.add(Dense(21, activation = 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X_lstm,train_y_lstm,validation_data = (train_X_lstm,train_y_lstm), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(test_X_lstm)
            previsoes = scaler.inverse_transform(previsoes)
            media_previsoes.append(myutils.mean_absolute_percentage_error(y_test[:len(previsoes)], previsoes))

        df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged),np.mean(media_previsoes),lags)

        
    
df_result

Coliformes Termotolerantes
Oxigênio Dissolvido
Turbidez
Fósforo Total
Sólido Total
DBO (5, 20)
Temperatura da Água
pH


,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,151,55.41,1
1,Random Forest,Coliformes Termotolerantes,151,55.40,1
2,MLP,Coliformes Termotolerantes,151,50.55,1
3,LSTM,Coliformes Termotolerantes,151,52.56,1
4,Regressão Linear,Coliformes Termotolerantes,151,41.82,2
...,...,...,...,...,...
315,LSTM,pH,179,1.54,9
316,Regressão Linear,pH,179,1.66,10
317,Random Forest,pH,179,1.69,10
318,MLP,pH,179,1.67,10


In [185]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/temporal_pad_lag.csv', index = True, header=True)

In [186]:
#predição espaço-temporal
print('predição espaço-temporal')

parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    
    DFmerged1 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  
    
    DFmerged2 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02090') &
                      (DFmerge['parametro'] == parameter)]

    
    
    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    #DFmerged1 = DFmerged1[DFmerged1['codigo_ponto'] == 'TIET02050']    
    DFmerged1 = clean_group_data(DFmerged1, 'valor')
    
    #DFmerged2 = DFmerged2[DFmerged2['codigo_ponto'] == 'TIET02090']    
    DFmerged2 = clean_group_data(DFmerged2, 'valor')
    
     
    
    DFmerged1['data_coleta'] = pd.to_datetime(DFmerged1['data_coleta'])
    DFmerged1.index = DFmerged1['data_coleta']    
    DFmerged1 = DFmerged1.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged1 = DFmerged1.loc['1979-01-31':]
       
    DFmerged2['data_coleta'] = pd.to_datetime(DFmerged2['data_coleta'])
    DFmerged2.index = DFmerged2['data_coleta']    
    DFmerged2 = DFmerged2.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged2 = DFmerged2.loc['1979-01-31':]
    
    #coloca o dataframes com o mesmo tamanho
    #DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    if (len(DFmerged1) > len(DFmerged2)):
        DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    else:
        DFmerged2 = DFmerged2.iloc[0:len(DFmerged1)]   
    
    
    DFmerged1 = DFmerged1.interpolate(method = 'pad')
    DFmerged2 = DFmerged2.interpolate(method = 'pad')
    
    DFmerged1[parameter] = tranformacao_log(DFmerged1[parameter])
    DFmerged2[parameter] = tranformacao_log(DFmerged2[parameter])
        
    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
    #seleciona os dados
    
    train1,train_previsao1, test1 = pegar_dados_coluna_predita_train_test(DFmerged1,0.70,0)
    y_test1 = test1
    
    train2,train_previsao2, test2 = pegar_dados_coluna_predita_train_test(DFmerged2,0.70,0)
    y_test2 = test2
        
    for lags in range(1,11):
        
        #normalização dos dados
        scaler = MinMaxScaler()
        train1,test1,testd1 = normalizacao(train1,test1)
        train1,test1,testd1 = normalizacao(train2,test2)
                
        #Prepara os dados de treinamento
        train_X1,train_y1 = prepara_dados(train1, lags,0)  
        train_X_lstm1 = train_X1
        train_y_lstm1 = train_y1
        
        train_X2,train_y2 = prepara_dados(train2, lags,0)  
        train_X_lstm2 = train_X2
        train_y_lstm2 = train_y2
                
        #Ajusta a dimensão de train_X 
        nsamples, nx, ny = train_X1.shape
        train_X1 = train_X1.reshape((nsamples,nx*ny))
        
        nsamples, nx, ny = train_X2.shape
        train_X2 = train_X2.reshape((nsamples,nx*ny))
    
        #Prepara os dados de teste
        entradas1 = DFmerged1[len(DFmerged1) - len(test1) - lags:].values
        entradas1 = scaler.fit_transform(entradas1)  
    
        test_X1 = []
        for i in range(lags, lags+len(test1)):        
            test_X1.append(entradas1[i-lags:i, 0:1])
        test_X1 = np.array(test_X1)
        
        test_X_lstm1 = test_X1
        
        entradas2 = DFmerged2[len(DFmerged2) - len(test2) - lags:].values
        entradas2 = scaler.fit_transform(entradas2)  
    
        test_X2 = []
        for i in range(lags, lags+len(test2)):        
            test_X2.append(entradas2[i-lags:i, 0:1])
        test_X2 = np.array(test_X2)
        
        test_X_lstm2 = test_X2
                
        #Ajusta a dimensão de test_X e train_X
        nsamples, nx, ny = test_X1.shape
        test_X1 = test_X1.reshape((nsamples,nx*ny))
        
        nsamples, nx, ny = test_X2.shape
        test_X2 = test_X2.reshape((nsamples,nx*ny))
                        
        '''
        dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
        dataset.columns = ['X','y']
        dataset.dropna(inplace=True)
        '''
                        
        #Regressão Linear
        model = LinearRegression(normalize=False)

        model.fit(train_X1,train_y1)
        
        #Dados de teste
        previsoes = model.predict(test_X2).reshape(-1, 1) 
        previsoes = scaler.inverse_transform(previsoes)
        
        #print('y_test:',len(y_test))
        #print('previsoes:',len(previsoes))
               
        df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_test2[:len(previsoes)], previsoes),lags)
        
        
        #Random Forest
        model = RandomForestRegressor()

        model.fit(train_X1,train_y1)
        
        #Dados de teste
        previsoes = model.predict(test_X2).reshape(-1, 1)
        previsoes = scaler.inverse_transform(previsoes)

        df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_test2[:len(previsoes)], previsoes),lags)
        
        #MLP
        model = Sequential()
        model.add(Dense(units = 10, activation = 'relu', input_dim = train_X1.shape[1]))
        model.add(Dense(units = 21, activation = 'relu'))
        model.add(Dense(units = 1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X1,train_y1, validation_data = (train_X2,train_y2),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(test_X2)
            previsoes = scaler.inverse_transform(previsoes)

            media_previsoes.append(myutils.mean_absolute_percentage_error(y_test2[:len(previsoes)], previsoes))  

        df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged),np.mean(media_previsoes),lags)

        
        #LSTM

        model = Sequential()
        model.add(LSTM(units = 10, input_shape = (train_X_lstm1.shape[1], 1)))
        model.add(Dense(21, activation = 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X_lstm1,train_y_lstm1,validation_data = (train_X_lstm2,train_y_lstm2), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(test_X_lstm2)
            previsoes = scaler.inverse_transform(previsoes)
            media_previsoes.append(myutils.mean_absolute_percentage_error(y_test2[:len(previsoes)], previsoes))

        df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged),np.mean(media_previsoes),lags)

        
    
df_result

predição espaço-temporal


,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,179,26.49,1
1,Random Forest,Coliformes Termotolerantes,179,29.12,1
2,MLP,Coliformes Termotolerantes,179,31.81,1
3,LSTM,Coliformes Termotolerantes,179,32.22,1
4,Regressão Linear,Coliformes Termotolerantes,179,25.84,2
...,...,...,...,...,...
315,LSTM,pH,179,0.86,9
316,Regressão Linear,pH,179,0.90,10
317,Random Forest,pH,179,0.87,10
318,MLP,pH,179,0.86,10


In [187]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/espaco_temporal_pad_lag.csv', index = True, header=True)

In [180]:
#predição espacial
print('predição espacial')

parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    
    DFmerged1 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  
    
    DFmerged2 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02090') &
                      (DFmerge['parametro'] == parameter)]

    
    
    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    #DFmerged1 = DFmerged1[DFmerged1['codigo_ponto'] == 'TIET02050']    
    DFmerged1 = clean_group_data(DFmerged1, 'valor')
    
    #DFmerged2 = DFmerged2[DFmerged2['codigo_ponto'] == 'TIET02090']    
    DFmerged2 = clean_group_data(DFmerged2, 'valor')
    
     
    
    DFmerged1['data_coleta'] = pd.to_datetime(DFmerged1['data_coleta'])
    DFmerged1.index = DFmerged1['data_coleta']    
    DFmerged1 = DFmerged1.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged1 = DFmerged1.loc['1979-01-31':]
       
    DFmerged2['data_coleta'] = pd.to_datetime(DFmerged2['data_coleta'])
    DFmerged2.index = DFmerged2['data_coleta']    
    DFmerged2 = DFmerged2.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged2 = DFmerged2.loc['1979-01-31':]
    
    #coloca o dataframes com o mesmo tamanho
    #DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    if (len(DFmerged1) > len(DFmerged2)):
        DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    else:
        DFmerged2 = DFmerged2.iloc[0:len(DFmerged1)]   
    
    DFmerged1 = DFmerged1.interpolate(method = 'pad')
    DFmerged2 = DFmerged2.interpolate(method = 'pad')
    
    DFmerged1[parameter] = tranformacao_log(DFmerged1[parameter])
    DFmerged2[parameter] = tranformacao_log(DFmerged2[parameter])
    
    scaler = MinMaxScaler()
    #DFmerged = scaler.fit_transform(np.log(DFmerged + 0.0000001))
#     DFmerged = scaler.fit_transform()

    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
        
    X  = DFmerged1[:]
    y  = DFmerged2[:]


    dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
    dataset.columns = ['X','y']
    dataset.dropna(inplace=True)

    #train_X = scaler.fit_transform(np.array(dataset['X']).reshape(len(dataset), 1))
    #train_y = scaler.fit_transform(np.log(np.array(dataset['y']).reshape(len(dataset), 1) + 0.000000001))
    
    train_X = scaler.fit_transform(np.array(dataset['X']).reshape(len(dataset), 1))
    train_y = scaler.fit_transform(np.array(dataset['y']).reshape(len(dataset), 1))
        
    #tamanho_treino = int(len(train_X)*0.7)
    tamanho_teste  = int(len(train_y)*0.3)

    #Desnormalização train_y(teste)
    y_teste = scaler.inverse_transform(train_y[-tamanho_teste:])

    #Regressão Linear
    model = LinearRegression(normalize=False)

    #model.fit(train_X[:-40],train_y[:-40])
    model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
    score = model.score(train_X,train_y)
    #     print(score)

    #Dados de teste
    #previsoes = model.predict(train_X[-40:])
    previsoes = model.predict(train_X[-tamanho_teste:])
    previsoes = scaler.inverse_transform(previsoes)

    #df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),myutils.mean_absolute_percentage_error(train_y[-40:], previsoes))
    df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged1),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),'')
        
        
    #print(parameter, len(DFmerged),'MAPE',str(myutils.mean_absolute_percentage_error(train_y[-40:], previsoes)))

    #Random Forest
    model = RandomForestRegressor()

    model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
    score = model.score(train_X,train_y)
    #     print(score)

    #Dados de teste
    previsoes = model.predict(train_X[-tamanho_teste:])
    previsoes = scaler.inverse_transform(previsoes.reshape(-1, 1))

    df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged1),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),'')

    #MLP
    model = Sequential()
    model.add(Dense(units = 10, activation = 'relu', input_dim = train_X[:-tamanho_teste].shape[1]))
    model.add(Dense(units = 21, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

    es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

    #Treina o modelo
    history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste], validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

    #Dados de teste
    media_previsoes = []
    for r in range(0,6):
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)

        media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))  

    df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged1),np.mean(media_previsoes),'')


    #LSTM

    train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
    train_y = np.reshape(train_y, (train_y.shape[0], train_y.shape[1], 1))


    model = Sequential()
    model.add(LSTM(units = 10, input_shape = (train_X[:-tamanho_teste].shape[1], 1)))
    model.add(Dense(21, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

    es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

    #Treina o modelo
    history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste],validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

    #Dados de teste
    media_previsoes = []
    for r in range(0,6):
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)
        media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))

    df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged1),np.mean(media_previsoes),'')

         
    
df_result

predição espacial


,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,151,23.96,
1,Random Forest,Coliformes Termotolerantes,151,29.00,
2,MLP,Coliformes Termotolerantes,151,30.62,
3,LSTM,Coliformes Termotolerantes,151,29.55,
4,Regressão Linear,Oxigênio Dissolvido,186,17.81,
5,Random Forest,Oxigênio Dissolvido,186,18.80,
6,MLP,Oxigênio Dissolvido,186,20.26,
7,LSTM,Oxigênio Dissolvido,186,19.95,
8,Regressão Linear,Turbidez,176,37.62,
9,Random Forest,Turbidez,176,38.67,


In [181]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/espacial_pad.csv', index = True, header=True)

In [156]:
def tranformacao_log(valores_parametro):
   
    valores_log = []
   
    for i in range(len(valores_parametro)):
        #print(np.log(np.array(valores_parametro[i]) + 0.000000001))
        valores_log.append(np.log(np.array(valores_parametro[i]) + 0.000000001))
       
    return valores_log

parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    DFmerged = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  

    

    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    #DFmerged = DFmerged[DFmerged['codigo_ponto'] == 'TIET02050']    
    DFmerged = clean_group_data(DFmerged, 'valor')
    
    DFmerged['data_coleta'] = pd.to_datetime(DFmerged['data_coleta'])
    DFmerged.index = DFmerged['data_coleta']    
    DFmerged = DFmerged.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged = DFmerged.loc['1979-01-31':]
    
    DFmerged.interpolate(method = 'pad')
    
      
    scaler = MinMaxScaler()
    DFmerged = tranformacao_log(DFmerged[parameter])
DFmerged

[1.8245492922123363,
 1.8137383761098738,
 1.7917594693947216,
 1.8908503720232293,
 1.8405496335562173,
 1.8562979905218762,
 1.8245492922123363,
 1.8245492922123363,
 1.8614928073981707,
 1.8562979905218762,
 1.8718021770554376,
 1.8562979905218762,
 1.8137383761098738,
 1.8082887713431999,
 1.8562979905218762,
 1.8245492922123363,
 1.8562979905218762,
 1.8082887713432,
 1.8405496335562173,
 1.8718021770554376,
 1.9315214117481414,
 1.8718021770554376,
 1.9169226123291199,
 1.8484548130620806,
 1.9021075265461742,
 1.8245492922123363,
 1.8562979905218762,
 1.9242486524201192,
 1.9021075265461742,
 1.8718021770554376,
 1.8405496335562173,
 1.8082887713431999,
 1.887069649183895,
 1.8405496335562173,
 1.8405496335562173,
 1.8245492922123363,
 1.8562979905218762,
 1.8082887713431999,
 1.7917594693947216,
 1.8405496335562173,
 1.8718021770554376,
 1.8325814639083102,
 1.8082887713431999,
 1.8245492922123363,
 1.8245492922123363,
 1.9021075265461742,
 1.8245492922123363,
 1.82454929221233